In [ ]:
import matplotlib.pyplot as plt
import torch

from torch.nn import ModuleList, BCEWithLogitsLoss
from torch.optim import Adam

from data import BinaryColoredMNIST

from models import Encoder, Classifier, Adversary

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

device = torch.device("cpu")
print(device)

### Data Modification

In [ ]:
data_dir = "data/mnist_data"
dataset1 = BinaryColoredMNIST(root=data_dir, alpha_odd=0.35, alpha_even=0.3)

In [ ]:
data = dataset1[:144]

plt.figure(figsize=(8, 8))

for i, (img, a, label) in enumerate(data):
    plt.subplot(12, 12, i + 1)
    plt.imshow(img.permute(1, 2, 0).numpy())
    plt.title(str(label.item()), fontsize=8)
    plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Setup for DP

C = 2
learning_rate = 1e-3
gamma = 1 / C

encoder = Encoder(10).to(device)
classifier = Classifier(10).to(device)
adv = Adversary(10).to(device)

adversaries = ModuleList([adv for _ in range(C)])

criterion_class = BCEWithLogitsLoss()

optimizer_enc_class = Adam(list(encoder.parameters()) + list(classifier.parameters()), lr=learning_rate)

optimizer_adv = Adam(adv.parameters(), lr=learning_rate / C)
optimizers_adv = [optimizer_adv for _ in range(C)]

train_loader  = torch.utils.data.DataLoader(
    dataset1, batch_size=64, shuffle=True, num_workers=4
)

In [ ]:
from train import train_laftr
train_laftr(encoder, classifier, adversaries, criterion_class, optimizer_enc_class, optimizers_adv, train_loader, gamma, device=device, verbose=True)